## Introduction

## The data

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip

In [7]:
!python preprocess_data.py -h

usage: preprocess_data.py [-h] -i INPUT -o OUTPUT [-t {gestures,locomotion}]

Preprocess OPPORTUNITY dataset

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        OPPORTUNITY zip file
  -o OUTPUT, --output OUTPUT
                        Processed data file
  -t {gestures,locomotion}, --task {gestures,locomotion}
                        Type of activities to be recognized


In [6]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t gestures

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

## Running DeepConvLSTM

### Setup

In [80]:
import lasagne
import theano
import time

import pandas as pd
import numpy as np
import pickle
import theano.tensor as T
import keras
from sklearn import metrics
from sliding_window import sliding_window
from keras import layers

# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113

# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 18

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# Batch Size
BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

### Load the sensor data

In [81]:
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
import collections

def load_dataset(filename):

    f = open(filename, 'rb')
    data = pickle.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('oppChallenge_locomotion.data')

def remove_nulls(X_train, y_train, X_test, y_test):
    uy_train = y_train[y_train > 0]
    uX_train = X_train[y_train > 0]
    uy_test = y_test[y_test > 0]
    uX_test = X_test[y_test > 0]

    uy_train[uy_train == 1] = 0
    uy_train[uy_train == 2] = 1
    uy_train[uy_train == 3] = 2
    uy_train[uy_train == 4] = 3

    uy_test[uy_test == 1] = 0
    uy_test[uy_test == 2] = 1
    uy_test[uy_test == 3] = 2
    uy_test[uy_test == 4] = 3
    
    return uX_train, uy_train, uX_test, uy_test


# RKN^, RKN_, BACK, HIP, R-SHOE, L-SHOE
# 53 Features

#features_delete = np.arange(6, 15) # 6,15
#features_delete = np.concatenate([features_delete, np.arange(6, 113)])
#features_delete = np.concatenate([features_delete, np.arange(21, 36)])
#features_delete = np.concatenate([features_delete, np.arange(45, 81)])

#X_train = np.delete(X_train, features_delete, 1)
#X_test = np.delete(X_test, features_delete, 1)

X_train, y_train, X_test, y_test = remove_nulls(X_train, y_train, X_test, y_test)

print(X_train.shape)
print(X_test.shape)

#unique, counts = np.unique(y_train, return_counts=True)
#print(dict(zip(unique, counts)))

Loading data...
 ..from file oppChallenge_locomotion.data
 ..reading instances: train (557963, 113), test (118750, 113)
(465668, 113)
(94260, 113)


In [66]:
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism

X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

#X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))
#X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]*X_test.shape[2]))

# Data is reshaped since the input of the network is a 4 dimension tensor
#X_train = X_train.reshape((-1, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
#X_test = X_test.reshape((-1, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))

print(X_train.shape)
print(X_test.shape)

 ..after sliding window (testing): inputs (7854, 24, 113), targets (7854,)
(38804, 24, 113)
(7854, 24, 113)


In [119]:
def mean_sliding_window(X):
    rows = X.shape[0]
    n_features = X.shape[2]
    window_mean = np.zeros((rows, n_features))
    for i in range(rows):
        window_mean[i] = np.mean(X_train[i],axis=0).reshape(1, n_features)
    return window_mean

s_X_train = mean_sliding_window(X_train)
s_X_test = mean_sliding_window(X_test)

print(s_X_train.shape)
print(s_X_test.shape)

(38804, 113)
(7854, 113)


### Define the Lasagne network

In [52]:
net = {}
net['input'] = lasagne.layers.InputLayer((BATCH_SIZE, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
net['conv1/5x1'] = lasagne.layers.Conv2DLayer(net['input'], NUM_FILTERS, (FILTER_SIZE, 1))
net['conv2/5x1'] = lasagne.layers.Conv2DLayer(net['conv1/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
net['conv3/5x1'] = lasagne.layers.Conv2DLayer(net['conv2/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
net['conv4/5x1'] = lasagne.layers.Conv2DLayer(net['conv3/5x1'], NUM_FILTERS, (FILTER_SIZE, 1))
net['shuff'] = lasagne.layers.DimshuffleLayer(net['conv4/5x1'], (0, 2, 1, 3))
net['lstm1'] = lasagne.layers.LSTMLayer(net['shuff'], NUM_UNITS_LSTM)
net['lstm2'] = lasagne.layers.LSTMLayer(net['lstm1'], NUM_UNITS_LSTM)
# In order to connect a recurrent layer to a dense layer, it is necessary to flatten the first two dimensions
# to cause each time step of each sequence to be processed independently (see Lasagne docs for further information)
net['shp1'] = lasagne.layers.ReshapeLayer(net['lstm2'], (-1, NUM_UNITS_LSTM))
net['prob'] = lasagne.layers.DenseLayer(net['shp1'],NUM_CLASSES, nonlinearity=lasagne.nonlinearities.softmax)
# Tensors reshaped back to the original shape
net['shp2'] = lasagne.layers.ReshapeLayer(net['prob'], (BATCH_SIZE, FINAL_SEQUENCE_LENGTH, NUM_CLASSES))
# Last sample in the sequence is considered
net['output'] = lasagne.layers.SliceLayer(net['shp2'], -1, 1)

### Load the model parameters

In [53]:
# The model is populated with the weights of the pretrained network

# all_params_values = pd.read_pickle(r'weights/DeepConvLSTM_oppChallenge_gestures.pkl') - old format of pickle.dump()
all_params_values = pickle.load(open('weights.pkl', 'rb'))

# save weights as pickle - pickle.dump(all_params_values ,open('weights.pkl' , 'wb' ))

lasagne.layers.set_all_param_values(net['output'], all_params_values)

### Run the model

In [54]:
# Compilation of theano functions
# Obtaining the probability distribution over classes
test_prediction = lasagne.layers.get_output(net['output'], deterministic=True)
# Returning the predicted output for the given minibatch
test_fn =  theano.function([ net['input'].input_var], [T.argmax(test_prediction, axis=1)])

In [63]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
# Classification of the testing data
print("Processing {0} instances in mini-batches of {1}".format(X_test.shape[0],BATCH_SIZE))
test_pred = np.empty((0))
test_true = np.empty((0))
start_time = time.time()
for batch in iterate_minibatches(X_test, y_test, BATCH_SIZE):
    inputs, targets = batch
    y_pred, = test_fn(inputs)
    test_pred = np.append(test_pred, y_pred, axis=0)
    test_true = np.append(test_true, targets, axis=0)

Processing 1654 instances in mini-batches of 100


In [64]:
# Results presentation
print("||Results||")
print("\tTook {:.3f}s.".format( time.time() - start_time))
import sklearn.metrics as metrics
print("\tTest fscore:\t{:.4f} ".format(metrics.f1_score(test_true, test_pred, average='weighted')))

||Results||
	Took 28.824s.
	Test fscore:	0.5317 


In [76]:
# kmeans with sliding window
# window is used to refit kmeans to change the positioning of the centroids

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors

size = 1000

asdf1 = X_train[:size,:,:]
asdf2 = X_test[:size,:,:]
asdf3 = y_test[:size]

kmeans = KMeans(n_clusters=4, random_state=3425)
for window in range(X_train.shape[0]):
    kmeans.fit(X_train[window])

In [78]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
def mostFrequent(arr, n): 
    # Sort the array 
    arr.sort() 
    # find the max frequency using 
    # linear traversal 
    max_count = 1; res = arr[0]; curr_count = 1
    for i in range(1, n):  
        if (arr[i] == arr[i - 1]): 
            curr_count += 1
        else: 
            if (curr_count > max_count):  
                max_count = curr_count 
                res = arr[i - 1]          
            curr_count = 1
    # If last element is most frequent 
    if (curr_count > max_count): 
        max_count = curr_count 
        res = arr[n - 1] 
    return res 

        
# Classification of the testing data
print("Processing {0} instances in mini-batches of {1}".format(X_test.shape[0], BATCH_SIZE))
test_pred = np.empty((0))
test_true = np.empty((0))
start_time = time.time()
for batch in iterate_minibatches(X_test, y_test, BATCH_SIZE):
    inputs, targets = batch
    # sliding window inputs are reduced to one dimension and then predicted
    y_pred = np.zeros((BATCH_SIZE,))
    for window in range(inputs.shape[0]):
        labels = kmeans.predict(inputs[window])
        label = mostFrequent(labels, inputs[window].shape[0]) # take mode of prediction
        y_pred[window] = label
    test_pred = np.append(test_pred, y_pred, axis=0)
    test_true = np.append(test_true, targets, axis=0)

Processing 7854 instances in mini-batches of 100


In [79]:
print(metrics.adjusted_rand_score(test_pred, test_true))

0.09219750230878679


In [82]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors

kmeans = KMeans(n_clusters=4, random_state=0).fit(X_train) # 5 with null & 4 without null
# gmm = GaussianMixture(n_components = 4, covariance_type='full').fit(X_train) # change number of iterations cause it didn't converge

y_pred = kmeans.predict(X_test)
#y_pred = spectral.fit_predict(X_test)

print(metrics.adjusted_rand_score(y_pred, y_test))

0.31033466772971036


In [83]:
pca = PCA(n_components=0.99)
pca.fit(X_train)
X_pca = pca.transform(X_test)
# print('at', x, '% of the variance ======')
print("Original shape:   ", X_test.shape)
print("Transformed shape:", X_pca.shape)

#df_redd = pd.DataFrame(X_pca)
#X_traind, X_testd, y_traind, y_testd = train_test_split(df_redd, y_train)

clf = KMeans(n_clusters=4, random_state=0).fit(X_pca)

y_pred = clf.predict(X_pca)

print(metrics.adjusted_rand_score(y_pred, y_test))

Original shape:    (94260, 113)
Transformed shape: (94260, 80)
0.3699421732152517


In [78]:
sliding_window = keras.Input(shape=(30, 113))

x = layers.Conv2D(112, (3, 3), activation='relu', padding='same')(sliding_window)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(112, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(512, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(sliding_window, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 1, 24, 113)]      0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 1, 24, 102)        103836    
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 1, 12, 102)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 1, 12, 86)         79034     
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 1, 6, 86)          0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 1, 6, 64)          49600     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 1, 3, 64)        

In [28]:
autoencoder1D = keras.Sequential(
    [
        layers.Input(shape=(30, 113)),
        layers.Conv1D(filters=64, kernel_size=7, padding="same", strides=15, activation="relu"),
        #layers.Dropout(rate=0.2),
        layers.Conv1D(filters=32, kernel_size=7, padding="same", strides=15, activation="relu"),
        layers.Conv1DTranspose(filters=32, kernel_size=7, padding="same", strides=15, activation="relu"),
        #layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(filters=64, kernel_size=7, padding="same", strides=15, activation="relu"),
        layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
    ]
)
autoencoder1D.compile(optimizer='adam', loss="mse")
autoencoder1D.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_17 (Conv1D)           (None, 2, 64)             512       
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 1, 32)             14368     
_________________________________________________________________
conv1d_transpose_23 (Conv1DT (None, 15, 32)            7200      
_________________________________________________________________
conv1d_transpose_24 (Conv1DT (None, 225, 64)           14400     
_________________________________________________________________
conv1d_transpose_25 (Conv1DT (None, 225, 1)            449       
Total params: 36,929
Trainable params: 36,929
Non-trainable params: 0
_________________________________________________________________


In [29]:
train_data = train_data.reshape(-1, 28,28, 1)
test_data = test_data.reshape(-1, 28,28, 1)

autoencoder1D.fit(X_train, X_train,
                epochs=50,
                batch_size=100,
                shuffle=True,
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Epoch 1/50


ValueError: in user code:

    C:\Users\ben_t\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\ben_t\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ben_t\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ben_t\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ben_t\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ben_t\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 

In [85]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
# should the autoencoder reduce the size of the sliding window?
# lstm autoencoder doesnt need a sliding window, redudent data can lead to overfitting

model = Sequential()
model.add(LSTM(108, activation='relu', input_shape=(30, 113), return_sequences=True))
model.add(LSTM(92, activation='relu', return_sequences=False))
model.add(Dense(86))
model.add(RepeatVector(30))
model.add(LSTM(92, activation='relu', return_sequences=True))
model.add(LSTM(108, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(113)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 108)            95904     
_________________________________________________________________
lstm_2 (LSTM)                (None, 92)                73968     
_________________________________________________________________
dense (Dense)                (None, 86)                7998      
_________________________________________________________________
repeat_vector (RepeatVector) (None, 30, 86)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 92)            65872     
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 108)           86832     
_________________________________________________________________
time_distributed (TimeDistri (None, 30, 113)          

In [107]:
from keras.callbacks import TensorBoard

asdf_T = X_train[:100,:,:]

model.fit(X_train, X_train, epochs=10, batch_size=100, callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Epoch 1/10
311/311 [==============================] - 43s 139ms/step - loss: 0.0057
Epoch 2/10
311/311 [==============================] - 40s 128ms/step - loss: 0.0037
Epoch 3/10
311/311 [==============================] - 39s 126ms/step - loss: 0.0030
Epoch 4/10
311/311 [==============================] - 39s 124ms/step - loss: 0.0027
Epoch 5/10
311/311 [==============================] - 38s 121ms/step - loss: 0.0024
Epoch 6/10
311/311 [==============================] - 39s 124ms/step - loss: 0.0023
Epoch 7/10
311/311 [==============================] - 40s 129ms/step - loss: 0.0021
Epoch 8/10
311/311 [==============================] - 39s 124ms/step - loss: 0.0020
Epoch 9/10
311/311 [==============================] - 39s 127ms/step - loss: 0.0020
Epoch 10/10
311/311 [==============================] - 40s 129ms/step - loss: 0.0019


In [108]:
# encoder becomes output
encoder = keras.Model(inputs=model.inputs, outputs=model.layers[2].output)

In [109]:
# get encoded data
encoded_train = encoder.predict(X_train, verbose=0)
encoded_test = encoder.predict(X_test, verbose=0)
print(encoded_train.shape)

(31043, 86)


In [81]:
#flattened_e_train = np.reshape(encoded_train,(encoded_train.shape[0],encoded_train.shape[1]*encoded_train.shape[2]))
#flattened_e_test = np.reshape(encoded_test,(encoded_test.shape[0],encoded_test.shape[1]*encoded_test.shape[2]))
print(flattened_encoded.shape)

flattened_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))
flattened_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]*X_test.shape[2]))

(6283, 2880)


In [111]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(encoded_train) # 5 with null & 4 without null

y_pred = kmeans.predict(encoded_test)

print(metrics.adjusted_rand_score(y_pred, y_test.reshape(y_test.shape[0])))

0.34250344732721577


In [78]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(encoded_train) # 5 with null & 4 without null

y_pred = kmeans.predict(encoded_test)

print(metrics.adjusted_rand_score(y_pred, y_test.reshape(y_test.shape[0])))

0.3174531239665948
